In [6]:
import random
import pandas as pd
import dash

# Create and populate a CSV file named "phone_sales.csv"
phone_names = ["iPhone 15", "Samsung Galaxy S24", "Google Pixel", "Nokia 3310", "Xiaomi Mi"]
columns = ['Phone Name', 'Contract', 'Account_up_to_date']

data = []
for i in range(500):
    phone_name = random.choice(phone_names)
    contract = random.choice([12, 24, 36])
    account_up_to_date = random.choice(["Yes", "No"])
    data.append([phone_name, contract, account_up_to_date])

# Save DataFrame to CSV 
df = pd.DataFrame(data, columns=columns)
csv_file_path = 'phone_sales.csv' 
df.to_csv(csv_file_path, index=False)

sales_df = pd.read_csv('phone_sales.csv')
sales_df = pd.DataFrame(sales_df)
sales_df


Phone Name  Contract Account_up_to_date
0            Nokia 3310        24                 No
1            Nokia 3310        24                Yes
2    Samsung Galaxy S24        36                 No
3            Nokia 3310        12                Yes
4    Samsung Galaxy S24        12                 No
..                  ...       ...                ...
495          Nokia 3310        12                 No
496           iPhone 15        12                Yes
497           iPhone 15        12                Yes
498           Xiaomi Mi        24                 No
499  Samsung Galaxy S24        36                 No

[500 rows x 3 columns]

In [8]:
import dash
from dash import dcc
from dash import html
import plotly.express as px


app = dash.Dash(__name__)

# Tab 1 layout
tab1 = html.Div([
    html.H1(children='Accounts', style={'textAlign': 'center', 'font-weight': 'bold'}),
    dcc.Dropdown(
        id='phone-dropdown',
        options=[{'label': phone, 'value': phone} for phone in sorted(sales_df['Phone Name'].unique())],
        value=sales_df['Phone Name'].iloc[0],
        clearable=False
    ),
    html.Div([
        dcc.RadioItems(
            id='contract-radio',
            options=[{'label': str(contract), 'value': contract} for contract in sorted(sales_df['Contract'].unique())],
            value=sales_df['Contract'].iloc[0],
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),
    dcc.Graph(id='account-status-bar')
])

@app.callback(
    dash.dependencies.Output('account-status-bar', 'figure'),
    [dash.dependencies.Input('phone-dropdown', 'value'),
     dash.dependencies.Input('contract-radio', 'value')]
)

def update_account_status(phone_name, contract_months):
    filtered_df = sales_df[(sales_df['Phone Name'] == phone_name) & (sales_df['Contract'] == contract_months)]
    account_counts = filtered_df['Account_UpToDate'].value_counts()
    fig = px.bar(x=account_counts.index, y=account_counts.values, 
                 labels={'x': 'Account Up To Date', 'y': 'Count'},
                 title=f'Account Up To Date for {phone_name} ({contract_months} months contract)')
    return fig




# Tab 2 layout
tab2 = html.Div([
    html.H1(children='Phone Contracts', style={'textAlign': 'center', 'font-weight': 'bold'}),
    dcc.Graph(id='phone-contracts-multi-bar')
])

app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Tab 1', children=tab1),
        dcc.Tab(label='Tab 2', children=tab2),
    ])
])


@app.callback(
    dash.dependencies.Output('phone-contracts-multi-bar', 'figure'),
    [dash.dependencies.Input('phone-dropdown', 'value')])

def update_phone_contracts(phone_name):
    filtered_df = sales_df[sales_df['Phone Name'] == phone_name]
    contract_counts = filtered_df['Contract'].value_counts().sort_index()
    fig = px.bar(x=contract_counts.index.astype(str), y=contract_counts.values, 
                 labels={'x': 'Contract Months', 'y': 'Number of Contracts'},
                 title=f'Contracts Taken for {phone_name}')
    fig.update_layout(hovermode='closest')
    fig.update_traces(hovertemplate='Phone: ' + phone_name + '<br>Contracts: %{y} <br>Months: %{x}')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


Dash is running on http://127.0.0.1:8050/



Dash is running on http://127.0.0.1:8050/



Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'


 * Debug mode: on
